In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat, Market Meet, Risk-Free)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [36]:
csv_name = "Tickers_Example.csv"
stocks = pd.read_csv(csv_name)

starting_money = 1000000

def isCanadian (ticker):
    return ticker.info['country'] == 'Canada'

us_ticker_ex = yf.Ticker("AAPL")
cad_ticker_ex = yf.Ticker("Shop.to")

def first_trading_days_of_each_month (start_date, end_date):
    us_ticker_ex_hist = us_ticker_ex.history(start= start_date, end = end_date)
    us_ticker_ex_hist.index = us_ticker_ex_hist.index.date
    cad_ticker_ex_hist = cad_ticker_ex.history(start= start_date, end = end_date)
    cad_ticker_ex_hist.index = cad_ticker_ex_hist.index.date

   
    valid_dates = np.intersect1d(us_ticker_ex_hist.index, cad_ticker_ex_hist.index)
    
    first_trading_days= []
    prev_month = 0
    for date in valid_dates:
        if date.month != prev_month:
            first_trading_days.append(date)
            prev_month = date.month

    return first_trading_days


dates_example = first_trading_days_of_each_month("2022-01-01", "2025-01-01")
dates_example

[datetime.date(2022, 1, 4),
 datetime.date(2022, 2, 1),
 datetime.date(2022, 3, 1),
 datetime.date(2022, 4, 1),
 datetime.date(2022, 5, 2),
 datetime.date(2022, 6, 1),
 datetime.date(2022, 7, 5),
 datetime.date(2022, 8, 2),
 datetime.date(2022, 9, 1),
 datetime.date(2022, 10, 3),
 datetime.date(2022, 11, 1),
 datetime.date(2022, 12, 1),
 datetime.date(2023, 1, 3),
 datetime.date(2023, 2, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 4, 3),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 6, 1),
 datetime.date(2023, 7, 5),
 datetime.date(2023, 8, 1),
 datetime.date(2023, 9, 1),
 datetime.date(2023, 10, 2),
 datetime.date(2023, 11, 1),
 datetime.date(2023, 12, 1),
 datetime.date(2024, 1, 2),
 datetime.date(2024, 2, 1),
 datetime.date(2024, 3, 1),
 datetime.date(2024, 4, 1),
 datetime.date(2024, 5, 1),
 datetime.date(2024, 6, 3),
 datetime.date(2024, 7, 2),
 datetime.date(2024, 8, 1),
 datetime.date(2024, 9, 3),
 datetime.date(2024, 10, 1),
 datetime.date(2024, 11, 1),
 datetime.da

In [39]:

# figure out how many shares of a stock you can buy given the price of 
# the stock and some amount of CAD and the current
# exchange rate (1 CAD = X USD)
def shares (ticker, price, money, exchange_rate):
    cad_transaction_fee_per_stock = 0.01/exchange_rate
    cad_max_transaction_fee = 2.15/exchange_rate
    cad_price = price

    if not isCanadian(ticker):
        cad_price = price/exchange_rate

    return max(money/(cad_price+cad_transaction_fee_per_stock),
               (money-cad_max_transaction_fee)/cad_price)


# this is for calculating monthly percentage returns of a portfolio of stocks
# also works for just one stock
def create_df(stocks, dates, distribution):
    start_date = dates[0]
    end_date = dates[len(dates)-1]+pd.Timedelta(days=1)
    
    # creating a dataframe for each ticker with its close price at every first trading day of the month
    stocks_df = []

    
    for ticker in stocks:
        temp_df = ticker.history(start=start_date, end=end_date)  
        temp_df.index = temp_df.index.date
        count = 0

        df_dict = {'Date': dates, 
               'Close': []}

        for i in range (len (temp_df)):
            if temp_df.index[i] == dates[count]:
                close = temp_df['Close'].iloc[i]
                df_dict['Close'].append(close)
                count += 1


        stock_df = pd.DataFrame(df_dict)
        stock_df = stock_df.set_index("Date")
      
        stocks_df.append(stock_df)

    # finding out how many shares in each stock will be bought
    shares_in_stocks = []

    money_in_each_stock = []
    for i in range (len(stocks)):
        money_in_each_stock.append(distribution[i]*starting_money)
    
    for i in range (len(stocks)):
        close = stocks_df[i]['Close'].iloc[0]
        shares_in_stocks.append(money_in_each_stock[i]/close)

    # creating a dataframe that stores the portfolio value at every first trading day of the month
    stock_portfolio = {
        "Date": dates,
        "Portfolio Value": []}
    
    for i in range (len(dates)):
        portfolio_value = 0
        for j in range(len(stocks)):
            close = stocks_df[j]['Close'].iloc[i]
            portfolio_value += close*shares_in_stocks[j]
            
        stock_portfolio["Portfolio Value"].append(portfolio_value)
    
    stock_portfolio = pd.DataFrame(stock_portfolio)
    stock_portfolio = stock_portfolio.set_index("Date")
    stock_portfolio['Percentage Returns']= stock_portfolio['Portfolio Value'].pct_change()*100
    stock_portfolio.drop(index=stock_portfolio.index[0], inplace = True)
    return stock_portfolio

# example usage
create_df([us_ticker_ex, cad_ticker_ex], dates_example, [0.1, 0.9])

,Portfolio Value,Percentage Returns
Date,,
2022-02-01,8.205513e+05,-17.944875
2022-03-01,5.886029e+05,-28.267384
2022-04-01,5.990723e+05,1.778686
2022-05-02,4.261946e+05,-28.857568
2022-06-01,3.481467e+05,-18.312729
2022-07-05,3.361961e+05,-3.432636
2022-08-02,3.710754e+05,10.374683
2022-09-01,3.268580e+05,-11.916013
2022-10-03,2.987676e+05,-8.594079
